### Visualize (Hand Coupling / Save Video)

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2R,r2quat,get_uv_dict_nc,get_p_target_nc
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig_psyonic.xml`

In [2]:
xml_path = '../asset/common_rig/scene_common_rig_psyonic.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0050] HZ:[200]
n_dof (=nv):[75]
n_geom:[65]
geom_names:['floor', 'base', 'root2spine', 'spine2neck', 'neck2rclavicle', 'neck2lclavicle', 'neck2head', 'rclavicle2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rthumb', 'rthumb_l12', 'rthumb_l23', 'rthumb_l3end', 'rthumb2rindex', 'rthumb2rmiddle', 'rthumb2rring', 'rthumb2rpinky', 'rindex_l12', 'rindex_l23', 'rindex_l3end', 'rmiddle_l12', 'rmiddle_l23', 'rmiddle_l3end', 'rring_l12', 'rring_l23', 'rring_l3end', 'rpinky_l12', 'rpinky_l23', 'rpinky_l3end', 'lclavicle2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lthumb', 'lthumb_l12', 'lthumb_l23', 'lthumb_l3end', 'lthumb2lindex', 'lthumb2lmiddle', 'lthumb2lring', 'lthumb2lpinky', 'lindex_l12', 'lindex_l23', 'lindex_l3end', 'lmiddle_l12', 'lmiddle_l23', 'lmiddle_l3end', 'lring_l12', 'lring_l23', 'lring_l3end', 'lpinky_l12', 'lpinky_l23', 'lpinky_l3end', 'head', 'nose', 'base2rpelvis', 'rpelvis2rknee', 'rknee2rankle', 'rankle', 'rfoot', 'base2lpelvis', 'lpelvis2lknee

### Visualization (Hand Coupling / Save Video)

In [15]:
import pickle
import os
import cv2

HAND_COUPLING = False
SAVE_VIDEO = True

pkl_path = "../data/pkl/VAAI_DIRECT_21_01_a_M1.pkl"

with open(pkl_path,'rb') as f:
    data = pickle.load(f)

video = []
tick = 0
L = data['length']

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,1.2])
env.reset()

while tick < L:
    q = data['qpos'][tick,:]
    p_root = data['root'][tick,:]

    if HAND_COUPLING:
        q_couple = q
        
        coupled_joint_idx_list = [[22,23],[24,25,26],[27,28,29],[30,31,32],[33,34,35],
                                  [45,46],[47,48,49],[50,51,52],[53,54,55],[56,57,58]]

        coupled_joint_weights_list = [[1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],
                                      [1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2]]

        for i in range(len(coupled_joint_idx_list)):
            coupled_joint_idx = coupled_joint_idx_list[i]
            coupled_joint_weights = coupled_joint_weights_list[i]
            joint_sum = 0

            for j in range(len(coupled_joint_idx)):
                joint_sum += q[coupled_joint_idx[j]]

            joint_sum /= np.sum(coupled_joint_weights)

            for k in range(len(coupled_joint_idx)):
                q_couple[coupled_joint_idx[k]] = joint_sum*coupled_joint_weights[k]

        q = q_couple
    
    env.forward(q=q,INCREASE_TICK=True)
    env.set_p_root(root_name='base',p=p_root)

    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        env.render()  
        frame = env.grab_image()
        video.append(frame)

        for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
            axis_joint = env.model.jnt_axis[rev_joint_idx]
            p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
            axis_world = R_joint@axis_joint
            axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
            axis_len,axis_r = 0.02,0.002
            env.plot_arrow_fr2to(
                p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
                r=axis_r,rgba=axis_rgba)

    tick = tick + 1
    if tick == L:
        if not SAVE_VIDEO: tick = 0

env.close_viewer()

if SAVE_VIDEO:
    shape = video[0].shape
    if HAND_COUPLING: 
        video_folder = ("../video/hand_coupled/") 
    else: 
        video_folder = ("../video/original/")
    if not os.path.isdir(video_folder): os.mkdir(video_folder)

    video_path = os.path.join(video_folder, pkl_path.split('/')[-1].split('.')[0] + ".mp4")
    video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
    fps = 50
    video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

    for i in range(len(video)):
        video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
        video_out.write(video_frame)
    video_out.release()

### Repeat for all 80 sequences

In [16]:
import numpy as np
import pickle
import cv2
import os

HAND_COUPLING = False
SAVE_VIDEO = True

data_types = ["a","b","c"]

for data_type in data_types:
    if data_type == "a":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "b":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "c":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    for data_number in data_numbers:
        data_name = "VAAI_DIRECT_" + data_number + "_" + data_type + "_M1"
        pkl_path = "../data/pkl/" + data_name + ".pkl"

        with open(pkl_path,'rb') as f:
            data = pickle.load(f)

        video = []
        tick = 0
        L = data['length']

        # Initialize MuJoCo viewer
        env.init_viewer(viewer_title='Common Rig H',viewer_width=1200,viewer_height=800,
                        viewer_hide_menus=True)
        env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,1.2])
        env.reset()

        while tick < L:
            q = data['qpos'][tick,:]
            p_root = data['root'][tick,:]

            if HAND_COUPLING:
                q_couple = q
                
                coupled_joint_idx_list = [[22,23],[24,25,26],[27,28,29],[30,31,32],[33,34,35],
                                  [45,46],[47,48,49],[50,51,52],[53,54,55],[56,57,58]]

                coupled_joint_weights_list = [[1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],
                                            [1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2]]

                for i in range(len(coupled_joint_idx_list)):
                    coupled_joint_idx = coupled_joint_idx_list[i]
                    coupled_joint_weights = coupled_joint_weights_list[i]
                    joint_sum = 0

                    for j in range(len(coupled_joint_idx)):
                        joint_sum += q[coupled_joint_idx[j]]

                    joint_sum /= np.sum(coupled_joint_weights)

                    for k in range(len(coupled_joint_idx)):
                        q_couple[coupled_joint_idx[k]] = joint_sum*coupled_joint_weights[k]

                q = q_couple
            
            env.forward(q=q,INCREASE_TICK=True)
            env.set_p_root(root_name='base',p=p_root)

            if env.loop_every(tick_every=1):
                # Plot world frame
                env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                        PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
                env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                        PLOT_AXIS=False,label="tick:[%d]"%(tick))
                env.render()  
                frame = env.grab_image()
                video.append(frame)

            tick = tick + 1

        env.close_viewer()

        if SAVE_VIDEO:
            shape = video[0].shape
            if HAND_COUPLING: 
                video_folder = ("../video/hand_coupled/") 
            else: 
                video_folder = ("../video/original/")
            if not os.path.isdir(video_folder): os.mkdir(video_folder)

            video_path = os.path.join(video_folder, pkl_path.split('/')[-1].split('.')[0] + ".mp4")
            video_fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
            fps = 50
            video_out = cv2.VideoWriter(video_path, video_fourcc, fps, (shape[1], shape[0]), True)

            for i in range(len(video)):
                video_frame = cv2.cvtColor(video[i], cv2.COLOR_BGR2RGB)
                video_out.write(video_frame)
            video_out.release()